##### Parte 1 > Realizando import's e iniciando pyspark

In [28]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext
from pyspark.sql.types import StringType
import random
from pyspark.sql.functions import *

In [29]:
# Nesta etapa, adicoine código para ler o arquivo nomes_aleatorios.txt através do comando spark.read.csv. 
# Carregue-o para dentro de um dataframe chamado df_nomes e, por fim, liste algumas linhas através do método show. Exemplo: df_nomes.show(5)

spark = SparkSession.builder.master("local[*]").appName("Tarefa").getOrCreate()
df_nomes = spark.read.csv('../Tarefa3/nomes_aleatorios.txt')
df_nomes.show(5)

+----------------+
|             _c0|
+----------------+
|  Frances Bennet|
|   Jamie Russell|
|  Edward Kistler|
|   Sheila Maurer|
|Donald Golightly|
+----------------+
only showing top 5 rows



##### Parte 2 > Renomear coluna, imprimir o schema e mostrar 10 linhas

In [30]:
df_nomes = df_nomes.withColumnRenamed('_c0', 'Nomes')
df_nomes.printSchema()
df_nomes.show(10)


root
 |-- Nomes: string (nullable = true)

+-----------------+
|            Nomes|
+-----------------+
|   Frances Bennet|
|    Jamie Russell|
|   Edward Kistler|
|    Sheila Maurer|
| Donald Golightly|
|       David Gray|
|      Joy Bennett|
|      Paul Kriese|
|Berniece Ornellas|
|    Brian Farrell|
+-----------------+
only showing top 10 rows



##### Parte 3 > Adicionar nova coluna chamada Escolaridade

In [31]:
escolaridade = ['Fundamental', 'Médio', 'Superior']

df_nomes = df_nomes.withColumn('Escolaridade', udf(f=lambda: escolaridade[random.randint(0,2)],returnType=StringType())())

##### Parte 4 > Adicionar países da América do Sul


In [32]:
paises = ["Brasil", "Argentina", "Colômbia", "Peru", "Venezuela", "Chile", "Equador",
            "Bolívia", "Paraguai", "Uruguai", "Guiana", "Suriname", "Guiana Francesa"]

df_nomes = df_nomes.withColumn('Pais', udf(f=lambda: paises[random.randint(0,12)],returnType=StringType())())

##### Parte 5 > Adicione nova coluna chamada AnoNascimento

In [33]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("CAST(FLOOR(RAND() * (66) + 1945) AS INT)"))

##### Parte 6 > Usando o método select do dataframe (df_nomes), selecione as pessoas que nasceram neste século

In [34]:
df_select = df_nomes.select('*').where(df_nomes.AnoNascimento >= 2000)
df_select.show(10)

+-----------------+------------+--------+-------------+
|            Nomes|Escolaridade|    Pais|AnoNascimento|
+-----------------+------------+--------+-------------+
|    Jamie Russell| Fundamental|Suriname|         2005|
|    Sheila Maurer|    Superior| Bolívia|         2010|
|Berniece Ornellas| Fundamental|Suriname|         2001|
|    Tracy Herring|       Médio|Colômbia|         2003|
|  Howard Lazarine|    Superior|   Chile|         2010|
|      Albert Leef| Fundamental|    Peru|         2000|
|   Gabriel Colyer|    Superior|    Peru|         2001|
|     Roxie Bernal| Fundamental|  Brasil|         2002|
|       John Meyer|    Superior| Equador|         2008|
|      Sandra Todd|       Médio|  Guiana|         2000|
+-----------------+------------+--------+-------------+
only showing top 10 rows



##### Parte 7 > Usando Spark SQL repita o processo da Pergunta 6.

In [35]:
df_nomes.createOrReplaceTempView('pessoas')
spark.sql('SELECT * FROM pessoas WHERE AnoNascimento >= 2000').show(10)


+-----------------+------------+---------------+-------------+
|            Nomes|Escolaridade|           Pais|AnoNascimento|
+-----------------+------------+---------------+-------------+
|    Jamie Russell|       Médio|        Bolívia|         2005|
|    Sheila Maurer|       Médio|         Brasil|         2010|
|Berniece Ornellas| Fundamental|       Paraguai|         2001|
|    Tracy Herring|    Superior|        Equador|         2003|
|  Howard Lazarine| Fundamental|       Suriname|         2010|
|      Albert Leef| Fundamental|         Guiana|         2000|
|   Gabriel Colyer|    Superior|Guiana Francesa|         2001|
|     Roxie Bernal| Fundamental|          Chile|         2002|
|       John Meyer| Fundamental|Guiana Francesa|         2008|
|      Sandra Todd|    Superior|        Equador|         2000|
+-----------------+------------+---------------+-------------+
only showing top 10 rows



##### Parte 8 > Conte o número de pessoas que são da geração Millennials


In [36]:
m = df_nomes.select('*').where(df_nomes.AnoNascimento.between(1980, 1994)).count()
print(m)


2274199


##### Parte 9 > Repita o processo da Pergunta 8 utilizando Spark SQL

In [37]:
spark.sql('SELECT COUNT(*) as Contagem FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994').show()

+--------+
|Contagem|
+--------+
| 2274199|
+--------+



##### Parte 10 > Obtenha a quantidade de pessoas de cada país para uma das gerações

In [49]:
sqlContext = SQLContext(spark)

babys = sqlContext.sql("SELECT Pais, 'Baby Boomers' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais")

ger_x = sqlContext.sql("SELECT Pais, 'Geração X' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

millenials = sqlContext.sql("SELECT Pais, 'Millennials' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

ger_z = sqlContext.sql("SELECT Pais, 'Geração Z' AS geracao, count(*) as Quantidade\
                        FROM pessoas\
                        WHERE AnoNascimento BETWEEN 1944 AND 1964\
                        GROUP BY Pais\
                        ORDER BY Pais, Geracao, Quantidade")

C:\Users\mathe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pyspark\sql\context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [51]:
babys.orderBy("Pais","Geracao", "Quantidade").show()

+---------------+------------+----------+
|           Pais|     geracao|Quantidade|
+---------------+------------+----------+
|      Argentina|Baby Boomers|    233427|
|        Bolívia|Baby Boomers|    233235|
|         Brasil|Baby Boomers|    232348|
|          Chile|Baby Boomers|    233394|
|       Colômbia|Baby Boomers|    233888|
|        Equador|Baby Boomers|    232753|
|         Guiana|Baby Boomers|    232731|
|Guiana Francesa|Baby Boomers|    233372|
|       Paraguai|Baby Boomers|    232996|
|           Peru|Baby Boomers|    232574|
|       Suriname|Baby Boomers|    233123|
|        Uruguai|Baby Boomers|    232799|
|      Venezuela|Baby Boomers|    232502|
+---------------+------------+----------+



In [52]:
ger_x.orderBy("Pais","Geracao", "Quantidade").show()

+---------------+---------+----------+
|           Pais|  geracao|Quantidade|
+---------------+---------+----------+
|      Argentina|Geração X|    233017|
|        Bolívia|Geração X|    233279|
|         Brasil|Geração X|    232593|
|          Chile|Geração X|    233194|
|       Colômbia|Geração X|    232941|
|        Equador|Geração X|    232966|
|         Guiana|Geração X|    232667|
|Guiana Francesa|Geração X|    232732|
|       Paraguai|Geração X|    232969|
|           Peru|Geração X|    233872|
|       Suriname|Geração X|    233290|
|        Uruguai|Geração X|    233493|
|      Venezuela|Geração X|    232129|
+---------------+---------+----------+



In [53]:
millenials.orderBy("Pais","Geracao", "Quantidade").show()

+---------------+-----------+----------+
|           Pais|    geracao|Quantidade|
+---------------+-----------+----------+
|      Argentina|Millennials|    233018|
|        Bolívia|Millennials|    232845|
|         Brasil|Millennials|    233166|
|          Chile|Millennials|    232669|
|       Colômbia|Millennials|    232942|
|        Equador|Millennials|    232715|
|         Guiana|Millennials|    232546|
|Guiana Francesa|Millennials|    233145|
|       Paraguai|Millennials|    233119|
|           Peru|Millennials|    233475|
|       Suriname|Millennials|    233783|
|        Uruguai|Millennials|    233138|
|      Venezuela|Millennials|    232581|
+---------------+-----------+----------+



In [54]:
ger_z.orderBy("Pais","Geracao", "Quantidade").show()

+---------------+---------+----------+
|           Pais|  geracao|Quantidade|
+---------------+---------+----------+
|      Argentina|Geração Z|    233008|
|        Bolívia|Geração Z|    232610|
|         Brasil|Geração Z|    233360|
|          Chile|Geração Z|    232555|
|       Colômbia|Geração Z|    232911|
|        Equador|Geração Z|    233788|
|         Guiana|Geração Z|    232744|
|Guiana Francesa|Geração Z|    232667|
|       Paraguai|Geração Z|    233230|
|           Peru|Geração Z|    233187|
|       Suriname|Geração Z|    233152|
|        Uruguai|Geração Z|    232856|
|      Venezuela|Geração Z|    233074|
+---------------+---------+----------+

